In [87]:
import os
import string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd


In [88]:
dt=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/SPAM-210331-134237.csv",encoding="ISO-8859-1")

In [89]:
data=np.array(dt)
# print(data.shape,data[0])
print(data[:5])
data[:, [0, 1]] = data[:, [1, 0]]
# print(data.shape,data[1])
print(data[:5])

[['ham'
  'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...']
 ['ham' 'Ok lar... Joking wif u oni...']
 ['spam'
  "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"]
 ['ham' 'U dun say so early hor... U c already then say...']
 ['ham' "Nah I don't think he goes to usf, he lives around here though"]]
[['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'
  'ham']
 ['Ok lar... Joking wif u oni...' 'ham']
 ["Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"
  'spam']
 ['U dun say so early hor... U c already then say...' 'ham']
 ["Nah I don't think he goes to usf, he lives around here though" 'ham']]


In [91]:
def preprocess_data(data):
    print("Preprocessing data...")
    
    punc = string.punctuation           # Punctuation list
    sw = stopwords.words('english')
    for record in data:
        # Remove common punctuation and symbols
      for item in punc:
        record[0] = record[0].replace(item, "")
         # Lowercase all letters and remove stopwords 
      splittedWords = record[0].split()
      newText = ""
      for word in splittedWords:
        if word not in sw:
          word = word.lower()
          newText = newText + " " + word
        
      record[0] = newText
        
    print("flag 2: preprocessed data")        
    return data
data=preprocess_data(data)
# print(data)

Preprocessing data...
flag 2: preprocessed data


In [103]:
def split_data(data):
    print("Splitting data...")
    
    features = data[:, 1]   # array containing all email text bodies
    labels = data[:, 0]     # array containing corresponding labels
    # print(labels)
    training_labels, test_labels, training_data, test_data =\
        train_test_split(features, labels, test_size = 0.27, random_state = 42)
    print(training_data[0],'|', test_data[0],'|', training_labels[0],'|', test_labels[0])
    print(len(training_data),'|', len(test_data),'|',len(training_labels),'|', len(test_labels))
    print("flag 3: splitted data")
    return training_data, test_data, training_labels, test_labels

In [93]:
def get_count(text):
    wordCounts = dict()
    for word in text.split():
        if word in wordCounts:
            wordCounts[word] += 1
        else:
            wordCounts[word] = 1
    
    return wordCounts

In [94]:
def euclidean_difference(test_WordCounts, training_WordCounts):
    total = 0
    for word in test_WordCounts:
      if word in test_WordCounts and word in training_WordCounts:
        total += (test_WordCounts[word] - training_WordCounts[word])**2
        del training_WordCounts[word]
      else:
        total += test_WordCounts[word]**2
    for word in training_WordCounts:
      total += training_WordCounts[word]**2
    return total**0.5

#To get spam count and ham count of k nearest neighbours.we selected k values
*italicized text*

In [95]:
def get_class(selected_Kvalues):
    spam_count = 0
    ham_count = 0
    for value in selected_Kvalues:
        if value[0] == "spam":
            spam_count += 1
        else:
            ham_count += 1
    if spam_count > ham_count:
        return "spam"
    else:
        return "ham"


In [96]:
def knn_classifier(training_data, training_labels, test_data, K, tsize):
    print("Running KNN Classifier...")
    
    result = []
    counter = 1
  # word counts for training email
    training_WordCounts = [] 
    for training_text in training_data:
            training_WordCounts.append(get_count(training_text))
    for test_text in test_data:
        similarity = [] # List of euclidean distances
        test_WordCounts = get_count(test_text)  # word counts for test email
        # Getting euclidean difference 
        for index in range(len(training_data)):
            euclidean_diff =\
                euclidean_difference(test_WordCounts, training_WordCounts[index])
            similarity.append([training_labels[index], euclidean_diff])
        # Sort list in ascending order based on euclidean difference
        similarity = sorted(similarity, key = lambda i:i[1])
        selected_Kvalues = [] 
        for i in range(K):
            selected_Kvalues.append(similarity[i])
        # Predicting the class of email
        result.append(get_class(selected_Kvalues))
    return result

In [106]:
def main(K):
    # data = load_data()
    # data = preprocess_data(data)
    training_data, test_data, training_labels, test_labels = split_data(data)
    tsize = len(test_data)
    result = knn_classifier(training_data, training_labels, test_data[:tsize], K, tsize)
    for i in range(tsize):
      print(test_labels[:tsize][i], result[i]) 
    accuracy = accuracy_score(test_labels[:tsize], result)
    print("training data size\t: " + str(len(training_data)))
    print("test data size\t\t: " + str(len(test_data)))
    print("K value\t\t\t\t: " + str(K))
    print("Samples tested\t\t: " + str(tsize))
    print("% accuracy\t\t\t: " + str(accuracy * 100))
    print("Number correct\t\t: " + str(int(accuracy * tsize)))
    print("Number wrong\t\t: " + str(int((1 - accuracy) * tsize)))
main(11)

Splitting data...
 sorry ill call later |  place man | ham | ham
84 | 32 | 84 | 32
flag 3: splitted data
Running KNN Classifier...
ham ham
ham ham
spam ham
ham ham
ham ham
ham ham
ham ham
ham ham
ham ham
spam ham
ham ham
ham ham
ham ham
ham ham
ham ham
ham ham
spam ham
ham ham
ham ham
ham ham
spam ham
ham ham
spam ham
spam ham
spam ham
spam ham
ham ham
ham ham
ham ham
ham ham
ham ham
spam ham
training data size	: 84
test data size		: 32
K value				: 11
Samples tested		: 32
% accuracy			: 71.875
Number correct		: 23
Number wrong		: 9
